In [6]:
import nltk
from nltk import pos_tag

from nltk.corpus import semcor
from nltk.corpus import wordnet as wn

import numpy as np

import networkx as nx


In [3]:
def similarity(synset1, synset2):
    meaning1 = synset1.definition()
    meaning2 = synset2.definition()
    set1 = meaning1.lower()
    set2 = meaning2.lower()
    s1List = set1.split(" ")
    s2List = set2.split(" ")
    count = 0
    for i in s1List:
        if (i in s2List):
            count += 1
    return count

In [4]:
sem_data = semcor.tagged_sents(tag = "sem")
pos_data = semcor.tagged_sents(tag = "pos")

In [5]:
finalWords = []
finalTags = []
finalSenses = []

totalSentences = 37176

for i in range(totalSentences):
    if (i%500 == 0):
        print(i)
    sent = sem_data[i]
    wordListInSentence = []
    posTagsInSentence = []
    sensesInSentence = []
    for j in range(len(sent)):
        if (isinstance(sent[j], nltk.tree.Tree) and isinstance(sent[j].label(), nltk.corpus.reader.wordnet.Lemma)):
            if (len(pos_data[i][j]) == 1):
                word = pos_data[i][j][0]
                wordListInSentence.append(word)
                posTagsInSentence.append(pos_data[i][j].label())
                sensesInSentence.append(sent[j].label().synset())

    finalWords.append(wordListInSentence)
    finalTags.append(posTagsInSentence)
    finalSenses.append(sensesInSentence)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000
36500
37000


In [ ]:
total = 0
matching = 0
for index in range(len(finalWords)):
    if (index%500 == 0 and index > 0):
        print(index, "Accuracy so far : ", matching/total)
    sentence_words = finalWords[index]
    num_synsets = []
    synset_to_index_maps = []
    all_synsets = []

    N = 0

    for i in sentence_words:
        synsets = wn.synsets(i)
        for synset in synsets:
            all_synsets.append(synset)
        wordmap = {}
        for i_dx, synset in enumerate(synsets):
            wordmap[synset] = i_dx
        synset_to_index_maps.append(wordmap)

        num_synsets.append(len(synsets))
        N += len(synsets)

    P = np.zeros((N, N))

    count = 0
    for i in range(N):
        for j in range(N):
            if (j != i):
                P[i][j] = similarity(all_synsets[i], all_synsets[j])

    for i in range(N):
        for j in range(N):
            if (np.sum(P[j]) > 0):
                P[i][j] = P[i][j]/np.sum(P[j])

    num_iter = 100
    beta = 0.85

    original_prob = np.ones((N, 1))/N
    final_prob = original_prob
    for i in range(num_iter):
        final_prob = np.matmul(P, final_prob)
        final_prob = beta*final_prob + (1-beta)/N
        final_prob = final_prob/np.linalg.norm(final_prob)

    final_prob = np.reshape(final_prob, (N, ))

    count = 0
    for i in range(len(sentence_words)):
        pos_of_word = finalTags[index][i]
        # print(pos_of_word)
        if (pos_of_word == "NN" or pos_of_word == "NNP" or pos_of_word == "NNS"):
            total += 1
            if (num_synsets[i] > 0):
                idx = np.argmax(final_prob[count : count + num_synsets[i]])
                sense_synset = all_synsets[count + idx]
                # print(sentence_words[i], sense_synset, finalSenses[index][i])
                if (sense_synset == finalSenses[index][i]):
                    matching += 1
        count += num_synsets[i]
    print(matching, total, matching/total)

print(total, matching)
print(matching/total)

In [15]:
all_senses_list = []
for i in finalSenses:
    for j in i:
        all_senses_list.append(j)

all_senses_set = set(all_senses_list)
total_unique_senses = len(all_senses_set)

sense_to_index = {}
index_to_sense = {}
count = 0
for i in all_senses_set:
    sense_to_index[i] = count
    index_to_sense[count] = i
    count += 1

confusion_matrix = np.zeros((total_unique_senses, total_unique_senses), dtype = np.int64)

In [5]:
total = 0
matching = 0
true = []
predicted = []
for index in range(len(finalWords)):
    if (index%500 == 0 and index > 0):
        print(index, "Accuracy so far : ", matching/total)
    sentence_words = finalWords[index]
    num_synsets = []
    synset_to_index_maps = []
    all_synsets = []

    N = 0

    for i in sentence_words:
        synsets = wn.synsets(i)
        for synset in synsets:
            all_synsets.append(synset)
        wordmap = {}
        for i_dx, synset in enumerate(synsets):
            wordmap[synset] = i_dx
        synset_to_index_maps.append(wordmap)

        num_synsets.append(len(synsets))
        N += len(synsets)

    D = nx.DiGraph()

    count = 0
    for i in range(N):
        for j in range(i):
            if (j != i):
                D.add_weighted_edges_from([(i, j, similarity(all_synsets[i], all_synsets[j]))])

    final_prob = nx.pagerank(D)
    final_prob = np.array([final_prob[key] for key in final_prob.keys()])

    count = 0
    if (len(final_prob) > 0):
        for i in range(len(sentence_words)):
            pos_of_word = finalTags[index][i]
            if (num_synsets[i] > 0):
                if (pos_of_word == "NN" or pos_of_word == "NNS" or pos_of_word == "NNP"):
                    total += 1
                    idx = np.argmax(final_prob[count : count + num_synsets[i]])
                    sense_synset = all_synsets[count + idx]
                    true.append(finalSenses[index][i])
                    predicted.append(sense_synset)
                    if (sense_synset == finalSenses[index][i]):
                        matching += 1
            count += num_synsets[i]

print(matching, total)

500 Accuracy so far :  0.4036894036894037
1000 Accuracy so far :  0.40994254309268047
1500 Accuracy so far :  0.4071394873407769
2000 Accuracy so far :  0.41194932000464957
2500 Accuracy so far :  0.41501441961067054
3000 Accuracy so far :  0.42273675127633564
3500 Accuracy so far :  0.4281849335658029
4000 Accuracy so far :  0.4369287964062581
4500 Accuracy so far :  0.44340276174202187
5000 Accuracy so far :  0.43656376727633867
5500 Accuracy so far :  0.43777876895628903
6000 Accuracy so far :  0.43916476574837693
6500 Accuracy so far :  0.4384251436296046
7000 Accuracy so far :  0.4379443031370247
7500 Accuracy so far :  0.43861398479371805
8000 Accuracy so far :  0.4375836530739716
8500 Accuracy so far :  0.43670067538439433
9000 Accuracy so far :  0.43567681589830415
9500 Accuracy so far :  0.43522224000853743
10000 Accuracy so far :  0.4345574144092816
10500 Accuracy so far :  0.434040940106141
11000 Accuracy so far :  0.43502382989123795
11500 Accuracy so far :  0.4354790453614

In [6]:
print("Accuracy", matching/total)
print("Precision", nltk.precision(set(true), set(predicted)))
print("Recall", nltk.recall(set(true), set(predicted)))
print("F1-Score", nltk.f_measure(set(true), set(predicted)))

Accuracy 0.43152201291212067
Precision 0.6694740187509932
Recall 0.7519186150276638
F1-Score 0.7083053127101547


In [23]:
new_total = 0
new_matching = 0

for index in range(len(finalWords)):
    sentence = finalWords[index]
    for i in range(len(sentence)):
        pos_of_word = finalTags[index][i]
        # if (pos_of_word == "NN" or pos_of_word == "NNS" or pos_of_word == "NNP"):
        if (len(wn.synsets(sentence[i])) > 0):
            if (wn.synsets(sentence[i])[0] == finalSenses[index][i]):
                new_matching += 1
        new_total += 1
print("Most Frequent sense (Wordnet First Sense) :", new_matching/new_total)

Most Frequent sense (Wordnet First Sense) : 0.4915068126915622


In [33]:
print("Input a sentence : ")
sentence = input()
print(sentence)

sentence_words = sentence.split()
tagged_sentence = nltk.pos_tag(sentence_words)
num_synsets = []
synset_to_index_maps = []
all_synsets = []

N = 0

for i in sentence_words:
    synsets = wn.synsets(i)
    for synset in synsets:
        all_synsets.append(synset)
    wordmap = {}
    for i_dx, synset in enumerate(synsets):
        wordmap[synset] = i_dx
    synset_to_index_maps.append(wordmap)

    num_synsets.append(len(synsets))
    N += len(synsets)

D = nx.DiGraph()

count = 0
for i in range(N):
    for j in range(i):
        if (j != i):
            D.add_weighted_edges_from([(i, j, similarity(all_synsets[i], all_synsets[j]))])

final_prob = nx.pagerank(D)
final_prob = np.array([final_prob[key] for key in final_prob.keys()])

count = 0
if (len(final_prob) > 0):
    for i in range(len(sentence_words)):
        pos_of_word = tagged_sentence[i][1]
        if (num_synsets[i] > 0):
            if (pos_of_word == "NN" or pos_of_word == "NNS" or pos_of_word == "NNP"):
                idx = np.argmax(final_prob[count : count + num_synsets[i]])
                sense_synset = all_synsets[count + idx]
                print(sentence_words[i], sense_synset, sense_synset.definition())

        count += num_synsets[i]

Input a sentence : 
I went to the bank to deposit some money, but it was closed today
bank Synset('bank.n.06') the funds held by a gambling house or the dealer in some gambling games
today Synset('today.n.01') the present time or age
